In [1]:
import array, struct, sys, os, tqdm
import numpy as np

def read_binaries(path_binfiles):
    result = {}
    cnt = 0
    for binfile in tqdm.tqdm(os.listdir(path_binfiles)):
        pdbcode = binfile.split('.')[0]                     # name of file (pdbcode)
        F = open('{0}/{1}'.format(path_binfiles, binfile), 'rb')
        n_decoys = struct.unpack('i', F.read(4))[0]         # number of decoys (=19 for this dataset)
        dimension = struct.unpack('i', F.read(4))[0]        # data dimensionality (23 protein types x 40 ligand types x 7 bins for this dataset)
        res = []
        for i in range(n_decoys):
            label = struct.unpack('d', F.read(8))[0]        # label (1 for native, -1 for non-native)
            data = array.array('d')                         
            data.fromfile(F, dimension)                     # feature vector (histograms, can be represented as a 23x40x7 matrix) 
            res.append([label, data])
        result[pdbcode] = res
        F.close()
        if cnt == 13080:
            break
        cnt += 1
    return result

In [2]:
result = read_binaries('../../../../../basic_experiment/general-no2013_t14_t3_l7.0_g1.0_r1.0')

100%|█████████▉| 13080/13089 [01:52<00:00, 116.08it/s]

In [3]:
with open('../../../../../basic_experiment/affinity_data_refined.csv', 'r') as f:
    data = f.read().split('\n')
    data = data[1:-1]

In [4]:
datasets = [
    {'name': d.split(',')[0], 'value': d.split(',')[1], 'type': d.split(',')[3]}
    for d in data
]

In [5]:
Kd_values = []
Ki_values = []
for d in datasets:
    if d['type'] == 'Kd':
        Kd_values.append(d)
    else:
        Ki_values.append(d)

In [6]:
Kd_data = []
for item in Kd_values:
    Kd_data.append([item['value']] + result[item['name']])

In [7]:
Ki_data = []
for item in Ki_values:
    if item['name'] != '966c':
        Ki_data.append([item['value']] + result[item['name']])

# Подбираем значение Cr по сетке

In [8]:
import time
import numpy as np
from math import log, exp
from scipy.linalg import sqrtm, inv, norm
from scipy.optimize import minimize

In [9]:
data = Ki_data

In [10]:
np.random.shuffle(data)
train = data[:int(len(data) * 0.7)]
test = data[int(len(data) * 0.7):]

100%|█████████▉| 13080/13089 [02:10<00:00, 100.60it/s]

In [11]:
# Матрица признаков (для которых аффинности известны)
X_nat_train = []
for i, t in enumerate(train):
    # Повышаем размерность (за счет вектора сдвигов b)
    additional = np.zeros(len(train))
    additional[i] = -1
    X_nat_train.append(np.append(t[1][1], additional))
X_nat_train = np.matrix(X_nat_train).T

# Столбец значений свободной энергии
s_train = np.matrix([
    float(t[0])
    for t in train
]).T

In [12]:
# Матрица признаков всей обучающей выборки
X_train = []
for i, t in enumerate(train):
    # Повышаем размерность (за счет вектора сдвигов b)
    additional = np.zeros(len(train))
    additional[i] = -1
    for pose in t[1:]:
        X_train.append(np.append(pose[1], additional))
        
X_train = np.matrix(X_train).T
print(X_train.shape)

# Классы обучающей выборки
y_train = []
for t in train:
    for pose in t[1:]:
        y_train.append(pose[0])

y_train = np.matrix(y_train).T

(7893, 27607)


In [14]:
# Матрица признаков всей тестовой выборки
X_test = []
for t in test:
    for pose in t[1:]:
        X_test.append(pose[1])
        
X_test = np.matrix(X_test).T

y_test = []
for t in test:
    for pose in t[1:]:
        y_test.append(pose[0])

y_test = np.matrix(y_test).T

In [ ]:
#Проходим по сетке значений Cr, записываем в файлы обратную матрицу A_inv, "новую" матрицу признаков X_new
for Cr in [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]:
    # Замена переменных
    XXT = X_nat_train @ X_nat_train.T
    I = np.identity(XXT.shape[0])
    A = np.real(sqrtm(0.5 * I + Cr * XXT))
    A_inv = inv(A)
    np.savetxt("grid_search/A_inv_cr_" + str(Cr), A_inv)
    
    newX_train = (A_inv.T @ X_train).T
    # Запись обучающей выборки в файл
    with open("grid_search/ki_train_cr_" + str(Cr), "w") as f:
        for i in range(newX_train.shape[0]):
            y_i = ("+1 " if y_train[i] == 1 else "-1 ")
            f.write(y_i)
            for j in range(newX_train.shape[1]):
                f.write(str(j + 1) + ":" + str(newX_train[i,j]) + " ")
            f.write("\n")
            
    print(Cr, "done")

# Тестирование

In [12]:
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [20]:
complexes_number = 831
poses_number = 19

# Матрица признаков (для которых аффинности известны)
X_test = np.matrix([
    t[1][1]
    for t in test
]).T

# Столбец значений свободной энергии
s_test = np.matrix([
    float(t[0])
    for t in test
]).T

Crs = [10]
pearsons = []
Rs = []
MSEs = []

for Cr in Crs:
    print("Cr =", Cr)
    
    # Считали полученный вектор w
    with open("cross_validation/ki_train_cr_" + str(Cr) + ".model", "r") as f:
        data = f.read().split("\n")
    newW = np.array(data[6:-1], dtype=float).reshape((8759, 1))
        
#     # Accuracy (poses):
#     newW = np.array(data[6:-1], dtype=float).reshape((6440, 1))
#     predicted_labels = (newW.T @ newX_test.T).tolist()[0]
#     y_pred = [
#        1 if label > 0 else -1
#        for label in predicted_labels
#     ]
#     cnt = 0
#     for i in range(complexes_number):
#        flag = True
#        for j in range(poses_number):
#            index = i * poses_number + j
#            if (y_pred[index] != y_test[index]):
#                flag = False
#        if flag is True:
#            cnt += 1
#     print("Accuracy: ", cnt / complexes_number)
    
    # Tests (affinities)
    
    # Считали матрицу A_inv
    A_inv = np.loadtxt("cross_validation/A_inv_cr_" + str(Cr))
    B = Cr * A_inv @ X_nat_train @ s_train
    # Обратная замена переменных
    w = A_inv @ (newW + B)
    # Отбрасываем лишние компоненты вектора w
    w = w[:6440]
    np.savetxt("cross_validation/ki_w_cr_" + str(Cr) + ".txt", w.T)
    prediction = w.T @ X_test
    print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
    pearsons.append(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0])[0])
    print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    Rs.append(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    MSEs.append(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    print()

Cr = 10
Spearman:  SpearmanrResult(correlation=0.46285137424377648, pvalue=5.7026302735672875e-83)
Pearson:  (0.46389138089287596, 2.2046558027070778e-83)
R2:  -107.114070715
MSE:  432.476786861



In [14]:
complexes_number = 831
poses_number = 19

# Матрица признаков (для которых аффинности известны)
X_test = np.matrix([
    t[1][1]
    for t in test
]).T

# Столбец значений свободной энергии
s_test = np.matrix([
    float(t[0])
    for t in test
]).T

Crs = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
pearsons = []
Rs = []
MSEs = []

for Cr in Crs:
    print("Cr =", Cr)
    
    # Считали полученный вектор w
    with open("cross_validation/ki_train_cr_" + str(Cr) + ".model", "r") as f:
        data = f.read().split("\n")
    newW = np.array(data[6:-1], dtype=float).reshape((7685, 1))
        
#     # Accuracy (poses):
#     newW = np.array(data[6:-1], dtype=float).reshape((6440, 1))
#     predicted_labels = (newW.T @ newX_test.T).tolist()[0]
#     y_pred = [
#        1 if label > 0 else -1
#        for label in predicted_labels
#     ]
#     cnt = 0
#     for i in range(complexes_number):
#        flag = True
#        for j in range(poses_number):
#            index = i * poses_number + j
#            if (y_pred[index] != y_test[index]):
#                flag = False
#        if flag is True:
#            cnt += 1
#     print("Accuracy: ", cnt / complexes_number)
    
    # Tests (affinities)
    
    # Считали матрицу A_inv
    A_inv = np.loadtxt("cross_validation/A_inv_cr_" + str(Cr))
    B = Cr * A_inv @ X_nat_train @ s_train
    # Обратная замена переменных
    w = A_inv @ (newW + B)
    # Отбрасываем лишние компоненты вектора w
    w = w[:6440]
    np.savetxt("cross_validation/ki_w_cr_" + str(Cr) + ".txt", w.T)
    prediction = w.T @ X_test
    print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
    pearsons.append(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0])[0])
    print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    Rs.append(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    MSEs.append(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    print()

Cr = 0.0001
Spearman:  SpearmanrResult(correlation=-0.409994762654802, pvalue=4.9844212622572388e-35)
Pearson:  (-0.36642333964928231, 8.3385954875318955e-28)
R2:  -12.3404078385
MSE:  59.8122104146

Cr = 0.001
Spearman:  SpearmanrResult(correlation=0.42690892792267837, pvalue=3.8978952709507938e-38)
Pearson:  (0.45654065886916034, 5.0705844321848694e-44)
R2:  -9.20978596959
MSE:  45.7759518371

Cr = 0.01
Spearman:  SpearmanrResult(correlation=0.52281583140347254, pvalue=1.7599610076904762e-59)
Pearson:  (0.52194026361430235, 2.9695640175696475e-59)
R2:  -2.8487372741
MSE:  17.2559554742

Cr = 0.1
Spearman:  SpearmanrResult(correlation=0.55754294493789058, pvalue=4.7323404641385009e-69)
Pearson:  (0.54529775975823924, 1.5069496502654572e-65)
R2:  -0.7887399757
MSE:  8.01988163323

Cr = 1
Spearman:  SpearmanrResult(correlation=0.5961224844960199, pvalue=4.2431698459141369e-81)
Pearson:  (0.57868069310121251, 1.8769662573830161e-75)
R2:  -0.249986565528
MSE:  5.60436085448

Cr = 10
Spear

In [40]:
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: MacOSX


In [47]:
plt.plot(np.log10(np.array(Crs)), pearsons, label="Pearson")
plt.xlabel(r"$\log_{10}{C_r}$", fontsize=17)
plt.ylabel(r"$\rho$", fontsize=17)
plt.show()

In [48]:
plt.plot(np.log10(np.array(Crs)), Rs, label="R^2")
plt.xlabel(r"$\log_{10}{C_r}$", fontsize=17)
plt.ylabel(r"$R^2$", fontsize=17)
plt.ylim([-14, 5])
plt.show()

In [50]:
plt.plot(np.log10(np.array(Crs)), MSEs, label="MSE")
plt.xlabel(r"$\log_{10}{C_r}$", fontsize=17)
plt.ylabel(r"$MSE$", fontsize=17)
plt.show()

# Cr = 100, подбираем значение C

In [54]:
A_inv = np.loadtxt("cross_validation/A_inv_cr_100")
B = Cr * A_inv @ X_nat_train @ s_train

In [55]:
complexes_number = 831
poses_number = 19

# Матрица признаков (для которых аффинности известны)
X_test = np.matrix([
    t[1][1]
    for t in test
]).T

# Столбец значений свободной энергии
s_test = np.matrix([
    float(t[0])
    for t in test
]).T

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
cpearsons = []
cRs = []
cMSEs = []
Cr = 100

for C in Cs:
    print("C =", C)
    
    with open("cross_validation/c_grid/ki_train_cr_100_c_" + str(C) + ".model", "r") as f:
        data = f.read().split("\n")
        
#     # Accuracy (poses):
#     newW = np.array(data[6:-1], dtype=float).reshape((6440, 1))
#     predicted_labels = (newW.T @ newX_test.T).tolist()[0]
#     y_pred = [
#        1 if label > 0 else -1
#        for label in predicted_labels
#     ]
#     cnt = 0
#     for i in range(complexes_number):
#        flag = True
#        for j in range(poses_number):
#            index = i * poses_number + j
#            if (y_pred[index] != y_test[index]):
#                flag = False
#        if flag is True:
#            cnt += 1
#     print("Accuracy: ", cnt / complexes_number)
    
    # Tests (affinities)
    newW = np.array(data[6:-1], dtype=float).reshape((7685, 1))
    w = A_inv @ (newW + B)
    w = w[:6440]
    np.savetxt("cross_validation/c_grid/ki_w_cr_100_c_" + str(C) + ".txt", w.T)
    prediction = w.T @ X_test
    print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
    pearsons.append(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0])[0])
    print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    Rs.append(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    MSEs.append(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    print()

C = 0.001
Spearman:  SpearmanrResult(correlation=0.60268060691746517, pvalue=2.5944127084992643e-83)
Pearson:  (0.58339392208761875, 6.045806517502942e-77)
R2:  0.0228384942097
MSE:  4.38113963989

C = 0.01
Spearman:  SpearmanrResult(correlation=0.60313116422808843, pvalue=1.8199735249424023e-83)
Pearson:  (0.58381501504663869, 4.4357253212559214e-77)
R2:  0.0229607455572
MSE:  4.38059152146

C = 0.1
Spearman:  SpearmanrResult(correlation=0.60674095512346204, pvalue=1.0411806655119487e-84)
Pearson:  (0.58674458871982138, 5.0797665217411782e-78)
R2:  0.0189372041506
MSE:  4.39863121771

C = 1
Spearman:  SpearmanrResult(correlation=0.61471896250691049, pvalue=1.636863636189424e-87)
Pearson:  (0.59379294455137632, 2.5222499679786557e-80)
R2:  -0.00604749529564
MSE:  4.51065103888

C = 10
Spearman:  SpearmanrResult(correlation=0.63159266419008164, pvalue=1.0278892726106344e-93)
Pearson:  (0.61649091751475804, 3.8052671639166052e-88)
R2:  0.0634068854142
MSE:  4.19924976213

C = 100
Spearma

In [64]:
complexes_number = 831
poses_number = 19

# Матрица признаков (для которых аффинности известны)
X_test = np.matrix([
    t[1][1]
    for t in test
]).T

# Столбец значений свободной энергии
s_test = np.matrix([
    float(t[0])
    for t in test
]).T

Cs = [10]
cpearsons = []
cRs = []
cMSEs = []
Cr = 100

for C in Cs:
    print("C =", C)
    
    with open("cross_validation/c_grid/ki_train_cr_100_c_" + str(C) + ".model", "r") as f:
        data = f.read().split("\n")
        
#     # Accuracy (poses):
#     newW = np.array(data[6:-1], dtype=float).reshape((6440, 1))
#     predicted_labels = (newW.T @ newX_test.T).tolist()[0]
#     y_pred = [
#        1 if label > 0 else -1
#        for label in predicted_labels
#     ]
#     cnt = 0
#     for i in range(complexes_number):
#        flag = True
#        for j in range(poses_number):
#            index = i * poses_number + j
#            if (y_pred[index] != y_test[index]):
#                flag = False
#        if flag is True:
#            cnt += 1
#     print("Accuracy: ", cnt / complexes_number)
    
    # Tests (affinities)
    newW = np.array(data[6:-1], dtype=float).reshape((7685, 1))
    w = A_inv @ (newW + B)
    w = w[:6440]
    np.savetxt("cross_validation/c_grid/ki_w_cr_100_c_" + str(C) + ".txt", w.T)
    prediction = w.T @ X_test
    print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
    pearsons.append(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0])[0])
    print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    Rs.append(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
    print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    MSEs.append(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))
    print()

C = 10
Spearman:  SpearmanrResult(correlation=0.70620604013164423, pvalue=1.8788398236755676e-126)
Pearson:  (0.69886889003492991, 8.7712281514396056e-123)
R2:  0.286293036243
MSE:  3.19993148691

